## Câu 1: Tính sự tương quan giữa A và B

In [1]:
import sqlite3

# Kết nối SQLite (sử dụng in-memory DB để test)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng Data với 2 cột A và B
cursor.execute("""
CREATE TABLE Data (
    A REAL,
    B REAL
);
""")

# Thêm dữ liệu ví dụ
cursor.executemany("INSERT INTO Data (A, B) VALUES (?, ?);", [
    (1, 2),
    (2, 4),
    (3, 6),
    (4, 8),
    (5, 10)
])

conn.commit()
result = cursor.execute("""
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) / 
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) * 
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS r_AB
FROM Data
""").fetchone()[0]

print("Hệ số tương quan Pearson r_AB =", result)


Hệ số tương quan Pearson r_AB = 0.9999999999999999


## Câu 2 Chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2

In [2]:
import sqlite3

# Tạo kết nối
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Bước 0: Tạo bảng dữ liệu
cursor.execute("CREATE TABLE Scores (Day TEXT, Sample TEXT, Score REAL)")
cursor.executemany("INSERT INTO Scores VALUES (?, ?, ?)", [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5),
])
conn.commit()

# Bước 1: Tính trung bình theo từng nhóm Sample
cursor.execute("""
CREATE TEMP VIEW group_means AS
SELECT Sample, AVG(Score) AS group_mean
FROM Scores
GROUP BY Sample;
""")

# Bước 2: Tính trung bình toàn bộ
overall_mean = cursor.execute("SELECT AVG(group_mean) FROM group_means").fetchone()[0]

# Bước 3: Tính số nhóm và tổng số điểm
num_groups = cursor.execute("SELECT COUNT(DISTINCT Sample) FROM Scores").fetchone()[0]
total_n = cursor.execute("SELECT COUNT(*) FROM Scores").fetchone()[0]

# Bước 4: Between Groups
between_groups = cursor.execute(f"""
SELECT SUM( (group_mean - {overall_mean}) * (group_mean - {overall_mean}) *
            (SELECT COUNT(*) FROM Scores WHERE Sample = group_means.Sample) )
FROM group_means;
""").fetchone()[0]
between_df = num_groups - 1
between_ms = between_groups / between_df

# Bước 5: Within Groups
within_groups = cursor.execute("""
SELECT SUM( (Score - group_mean) * (Score - group_mean) )
FROM Scores JOIN group_means USING (Sample);
""").fetchone()[0]
within_df = total_n - num_groups
within_ms = within_groups / within_df

# Bước 6: F-ratio
f_ratio = between_ms / within_ms

# In kết quả
print("Between Groups SS =", between_groups)
print("Within Groups SS  =", within_groups)
print("F-ratio =", f_ratio)



Between Groups SS = 1.5416666666666667
Within Groups SS  = 12.625
F-ratio = 0.5495049504950495


Nhận xét

- F-ratio = 0.55 nhỏ hơn rất nhiều so với giá trị tới hạn F ≈ 4.26 (nếu tra bảng với mức ý nghĩa α = 0.05).

- Điều này có nghĩa: khác biệt trung bình giữa các mẫu nhỏ hơn so với biến động bên trong từng mẫu.

- Do đó, không có bằng chứng thống kê đủ mạnh để khẳng định rằng các mẫu A, B, C khác nhau một cách đáng kể về điểm số.

- Không có sự khác biệt đáng kể giữa các mẫu (A, B, C) dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm.

In [3]:
import pandas as pd
from scipy.stats import chi2_contingency
# Chuyển sang Pandas để phân nhóm điểm
df = pd.read_sql_query("SELECT * FROM Scores", conn)

# Phân loại điểm thành nhóm
def score_group(score):
    if score >= 8:
        return 'High'
    elif score >= 6.5:
        return 'Medium'
    else:
        return 'Low'

df["ScoreGroup"] = df["Score"].apply(score_group)

# Tạo bảng đếm chéo (contingency table): ScoreGroup vs Sample
table_sample = pd.crosstab(df["Sample"], df["ScoreGroup"])
chi2_sample, p_sample, _, _ = chi2_contingency(table_sample)

# Tạo bảng đếm chéo: ScoreGroup vs Day
table_day = pd.crosstab(df["Day"], df["ScoreGroup"])
chi2_day, p_day, _, _ = chi2_contingency(table_day)

# In kết quả
print("Chi-Square theo MẪU XE:")
print(table_sample)
print(f"Chi2 = {chi2_sample:.3f}, p-value = {p_sample:.4f}")

print("\nChi-Square theo NGÀY:")
print(table_day)
print(f"Chi2 = {chi2_day:.3f}, p-value = {p_day:.4f}")

Chi-Square theo MẪU XE:
ScoreGroup  High  Low  Medium
Sample                       
A              1    1       2
B              2    1       1
C              1    1       2
Chi2 = 0.900, p-value = 0.9246

Chi-Square theo NGÀY:
ScoreGroup  High  Low  Medium
Day                          
Day 1          2    0       1
Day 2          1    0       2
Day 3          1    1       1
Day 4          0    2       1
Chi2 = 6.267, p-value = 0.3940


- p-value = 0.9246 > 0.05, điều này không có ý nghĩa thống kê.
- p-value = 0.3940 > 0.05, điều này cũng không có ý nghĩa thống kê

Không có sự phụ thuộc đáng kể giữa điểm số và yếu tố nào cả — không phải do loại mẫu xe, cũng không phải do ngày thử nghiệm.
Sự khác biệt điểm số trong bảng có thể là do yếu tố ngẫu nhiên, không có yếu tố cố định nào giải thích sự biến động.

## Câu 3 Chuyển đổi các giá trị này thành định dạng thời gian.

In [4]:
import sqlite3

# Kết nối và tạo bảng flights mẫu
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
);
""")

# Thêm dữ liệu mẫu
cursor.executemany("INSERT INTO flights (departure_time) VALUES (?);", [
    (830,), (1445,), (50,), (1210,), (0,)
])

conn.commit()

# Truy vấn chuyển đổi departure_time sang định dạng HH:MM
cursor.execute("""
SELECT 
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights;
""")

# In kết quả
for row in cursor.fetchall():
    print(row)


(830, '08:30')
(1445, '14:45')
(50, '00:50')
(1210, '12:10')
(0, '00:00')


## Câu 4  Tìm các ngoại lệ bằng cách sử dụng MAD

In [5]:
import sqlite3
import statistics

# Tạo SQLite in-memory
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng scores
cursor.execute("CREATE TABLE scores (score REAL);")
cursor.executemany("INSERT INTO scores (score) VALUES (?);", [
    (10,), (12,), (13,), (14,), (100,), (11,), (13,), (12,)
])
conn.commit()



In [6]:
# Lấy toàn bộ dữ liệu
cursor.execute("SELECT score FROM scores")
data = [row[0] for row in cursor.fetchall()]

# Tính median
med = statistics.median(data)

# Tính MAD: median(abs(x - median))
mad = statistics.median([abs(x - med) for x in data])

print(f"Median = {med}, MAD = {mad}")


Median = 12.5, MAD = 1.0


In [7]:
x = 1.5  # Hệ số phát hiện ngoại lệ

cursor.execute("""
SELECT *
FROM scores
WHERE ABS(score - ?) > (? * ?)
""", (med, x, mad))

# In kết quả ngoại lệ
print("Outliers:")
for row in cursor.fetchall():
    print(row)


Outliers:
(10.0,)
(100.0,)


## Câu 5 xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không

In [8]:
import sqlite3

# Kết nối tới cơ sở dữ liệu (ở đây là trong bộ nhớ, bạn có thể thay bằng tên file nếu có)
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng Patient
cur.execute('''
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    weight REAL,
    height REAL
)
''')

# Thêm dữ liệu mẫu
patients = [
    ('Nguyen', 60, 165),
    ('Nguyen', 60, 170),
    ('Tran', 55, 160),
    ('Nguyen', 62, 165),
    ('Tran', 55, 162)
]

cur.executemany('INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)', patients)
conn.commit()

# Tìm các cặp bệnh nhân có cùng họ và cân nặng
query = '''
SELECT p1.id, p2.id, p1.last_name, p1.weight
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
WHERE p1.last_name = p2.last_name AND p1.weight = p2.weight
'''

cur.execute(query)
matches = cur.fetchall()

# Hiển thị các cặp được cho là "cùng một người"
for match in matches:
    print(f"Cặp ID {match[0]} và {match[1]} có thể là cùng một người: last_name = {match[2]}, weight = {match[3]}")


Cặp ID 1 và 2 có thể là cùng một người: last_name = Nguyen, weight = 60.0
Cặp ID 3 và 5 có thể là cùng một người: last_name = Tran, weight = 55.0
